# SQUAD RU

**QA on SQuAD** - это задача найти ответ на вопрос в заданном контексте (например, абзац из Википедии), где ответ на каждый вопрос является находится в контексте

## Imports

In [ ]:
# !pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/RuslanKorchenov/transformers_my
!pip install git+https://github.com/huggingface/datasets.git
!pip install git-lfs
!git config --global user.email "ruselkomp1@gmail.com"
!git config --global user.name "ruselkomp"
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!pip install git+https://github.com/deepmipt/DeepPavlov.git

from huggingface_hub import notebook_login

import pandas as pd
import numpy as np

np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
import json

Для сохранения обученных моделей мы будем выгружать их на HuggingFace

In [ ]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


## Загрузка и обработка данных

Все используемые дальше данные доступны по ссылке: [Google Drive](https://drive.google.com/drive/folders/10fQmZAPIpGB4nEzESMu6oBsAPvi_zQjC?usp=sharing)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Заменяем роли на их index-ы


In [ ]:
!cp /content/drive/MyDrive/XLnet-tests/SberQuAD/train_roles.tsv train_roles.tsv
!cp /content/drive/MyDrive/XLnet-tests/SberQuAD/dev_roles.tsv dev_roles.tsv

In [ ]:
import pandas as pd

train_roles = pd.read_csv('train_roles.tsv', sep='\t')
dev_roles = pd.read_csv('dev_roles.tsv', sep='\t')

In [ ]:
train_roles

,context,context_roles,question,question_roles,answer,answer_start
0,В допетровское время искусство в России имело ...,"<unk>,<unk>,срок,тема,<unk>,<unk>,<pred>,<unk>...",Чем на Руси ограничивались проявления пластики?,"<unk>,<unk>,место,<pred>,тема,<unk>,<unk>","литьем небольших крестов, образов-складней, вы...",583
1,По данным рейтинга Top 100 самых филиальных ба...,"<unk>,<unk>,тема,<unk>,<unk>,<unk>,<unk>,<unk>...",Какое место занимает УБРиР по даным рейтинга T...,"<unk>,результат,<pred>,тема,<unk>,<unk>,<unk>,...",9 место,115
2,Смарт-стекло — современный собирательный терми...,"агенс,<unk>,эталон,<unk>,<unk>,<unk>,результат...",Какие свойства в общем способно изменять смарт...,"<unk>,<unk>,<unk>,<unk>,<unk>,<pred>,пациенс,<...",оптические,205
3,Во время интервенции в Россию Речи Посполитой ...,"<unk>,<unk>,агенс,<unk>,<unk>,<unk>,<unk>,мест...","Как звали воеводу Дорогобужа, который сдал гор...","<unk>,<pred>,адресат,<unk>,<unk>,агенс,<pred>,...",Иван Ададуров,287
4,В начале 1823 года Грибоедов на время покинул ...,"<unk>,сфера,<unk>,<unk>,агенс,<unk>,<unk>,<pre...",В каком году Грибоедов написал водевиль Кто бр...,"<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk...",1823,9
...,...,...,...,...,...,...
45312,Строительство военных линий стало опорной базо...,"агенс,<unk>,тема,<pred>,<unk>,статус,<unk>,<un...",Какая протяженность линии с юго-западной сторо...,"<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk...",553 версты,187
45313,Остаток жизни Диоклетиан провёл на родине в Ил...,"срок,<unk>,агенс,<pred>,<unk>,место,<unk>,коне...",В каком году умер Диоклетиан?,"<unk>,<unk>,эталон,<pred>,пациенс,<unk>",в 313 г.,680
45314,"Любимыми композиторами Толстого были Бах, Генд...","<unk>,цель,агенс,<unk>,<unk>,<unk>,<unk>,<unk>...",В каком году Лев Николаевич поселил у себя в Я...,"<unk>,<unk>,пациенс,<unk>,агенс,<pred>,<unk>,и...",В 1849,319
45315,Царство животных было разделено Линнеем на шес...,"пациенс,<unk>,<unk>,<pred>,агенс,<unk>,<unk>,р...",Что значит мусорным таксоном,"пациенс,<pred>,<unk>,способ","то есть систематической группой, составленной ...",632


In [ ]:
roles = train_roles['context_roles']

In [ ]:
def get_uniq(roles):
  uniq_roles = []

  for role in roles:
    r = role.split(',')
    for rol in r:
      if rol in uniq_roles:
        continue
      else:
        uniq_roles.append(rol)
  return uniq_roles

In [ ]:
uniq_context = get_uniq(train_roles['context_roles'])
uniq_context.sort()

In [ ]:
len(uniq_context)

46

In [ ]:
def get_ids(roles, uniq_roles):
  roles_ids = []

  for role in roles:
    r = role.split(',')
    temp = []
    for rol in r:
      for num, uniq in enumerate(uniq_roles):
        if uniq == rol:
          temp.append(num)
          break
    
    roles_ids.append(temp)

  return roles_ids

In [ ]:
test = get_ids(train_roles['context_roles'], uniq_context)

In [ ]:
test_q = get_ids(train_roles['question_roles'], uniq_context)

In [ ]:
train_roles['context_label'] = test

In [ ]:
train_roles['question_label'] = test_q

In [ ]:
train_roles

,context,context_roles,question,question_roles,answer,answer_start,context_label,question_label
0,В допетровское время искусство в России имело ...,"<unk>,<unk>,срок,тема,<unk>,<unk>,<pred>,<unk>...",Чем на Руси ограничивались проявления пластики?,"<unk>,<unk>,место,<pred>,тема,<unk>,<unk>","литьем небольших крестов, образов-складней, вы...",583,"[1, 1, 30, 41, 1, 1, 0, 1, 31, 0, 1, 1, 40, 1,...","[1, 1, 12, 0, 41, 1, 1]"
1,По данным рейтинга Top 100 самых филиальных ба...,"<unk>,<unk>,тема,<unk>,<unk>,<unk>,<unk>,<unk>...",Какое место занимает УБРиР по даным рейтинга T...,"<unk>,результат,<pred>,тема,<unk>,<unk>,<unk>,...",9 место,115,"[1, 1, 41, 1, 1, 1, 1, 1, 1, 1, 1, 1, 41, 2, 1...","[1, 24, 0, 41, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,Смарт-стекло — современный собирательный терми...,"агенс,<unk>,эталон,<unk>,<unk>,<unk>,результат...",Какие свойства в общем способно изменять смарт...,"<unk>,<unk>,<unk>,<unk>,<unk>,<pred>,пациенс,<...",оптические,205,"[2, 1, 44, 1, 1, 1, 24, 1, 1, 1, 1, 1, 1, 1, 1...","[1, 1, 1, 1, 1, 0, 14, 1, 1, 1]"
3,Во время интервенции в Россию Речи Посполитой ...,"<unk>,<unk>,агенс,<unk>,<unk>,<unk>,<unk>,мест...","Как звали воеводу Дорогобужа, который сдал гор...","<unk>,<pred>,адресат,<unk>,<unk>,агенс,<pred>,...",Иван Ададуров,287,"[1, 1, 2, 1, 1, 1, 1, 12, 1, 1, 1, 24, 24, 1, ...","[1, 0, 3, 1, 1, 2, 0, 14, 1, 0, 24, 1, 1, 1, 1..."
4,В начале 1823 года Грибоедов на время покинул ...,"<unk>,сфера,<unk>,<unk>,агенс,<unk>,<unk>,<pre...",В каком году Грибоедов написал водевиль Кто бр...,"<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk...",1823,9,"[1, 40, 1, 1, 2, 1, 1, 0, 14, 1, 0, 1, 8, 1, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...,...
45312,Строительство военных линий стало опорной базо...,"агенс,<unk>,тема,<pred>,<unk>,статус,<unk>,<un...",Какая протяженность линии с юго-западной сторо...,"<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk...",553 версты,187,"[2, 1, 41, 0, 1, 31, 1, 1, 1, 1, 1, 1, 0, 1, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
45313,Остаток жизни Диоклетиан провёл на родине в Ил...,"срок,<unk>,агенс,<pred>,<unk>,место,<unk>,коне...",В каком году умер Диоклетиан?,"<unk>,<unk>,эталон,<pred>,пациенс,<unk>",в 313 г.,680,"[30, 1, 2, 0, 1, 12, 1, 8, 1, 1, 1, 1, 1, 1, 1...","[1, 1, 44, 0, 14, 1]"
45314,"Любимыми композиторами Толстого были Бах, Генд...","<unk>,цель,агенс,<unk>,<unk>,<unk>,<unk>,<unk>...",В каком году Лев Николаевич поселил у себя в Я...,"<unk>,<unk>,пациенс,<unk>,агенс,<pred>,<unk>,и...",В 1849,319,"[1, 43, 2, 1, 1, 1, 1, 1, 1, 1, 41, 1, 1, 1, 2...","[1, 1, 14, 1, 2, 0, 1, 6, 1, 1, 12, 1, 1, 1]"
45315,Царство животных было разделено Линнеем на шес...,"пациенс,<unk>,<unk>,<pred>,агенс,<unk>,<unk>,р...",Что значит мусорным таксоном,"пациенс,<pred>,<unk>,способ","то есть систематической группой, составленной ...",632,"[14, 1, 1, 0, 2, 1, 1, 24, 1, 12, 1, 0, 1, 1, ...","[14, 0, 1, 29]"


In [ ]:
print(train_roles['context_label'][45316])

[1, 1, 43, 1, 12, 0, 1, 41, 1, 1, 0, 41, 1, 2, 1, 0, 29, 1, 43, 1, 1, 1, 8, 1, 1, 1, 41, 1, 8, 0, 1, 1, 13, 1, 1, 1, 1, 1, 14, 0, 29, 1, 1, 14, 41, 0, 1, 1, 1, 2, 1, 1, 0, 1, 8, 1, 1, 1, 41, 1, 0, 24, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
uniq_context = get_uniq(dev_roles['context_roles'])
uniq_context.sort()

In [ ]:
test = get_ids(dev_roles['context_roles'], uniq_context)

In [ ]:
test_q = get_ids(dev_roles['question_roles'], uniq_context)

In [ ]:
dev_roles['context_label'] = test

In [ ]:
dev_roles['question_label'] = test_q

## Переводим все данные в формат Datasets

In [ ]:
def convert_answers(row):
    return {'answer_start': [row['answer_start']], 'text': [row['answer']]}

train_t = train_roles
test_t = dev_roles

train_t['answers'] = train_t[['answer_start', 'answer']].apply(convert_answers, axis=1)
test_t['answers'] = test_t[['answer_start', 'answer']].apply(convert_answers, axis=1)

In [ ]:
len(train_t)

45317

In [ ]:
train_t = train_t.drop(columns = ['answer', 'answer_start'])
test_t = test_t.drop(columns = ['answer', 'answer_start'])

In [ ]:
train_t.drop(labels = [783,1386,2190,2826,2909,3844,3919,4192,6486,6786,7874,7918,8022,8110,8382,9287,9317,10887,11628,12136,12335,12684,14091,14352,14797,15006,15094,15556,15699,15709,16148,16282,16299,16363,16432,16656,16685,17321,17611,17861,17967,18940,19579,19959,19963,21638,22214,22989,25334,25474,25836,26836,27241,27642,27957,28008,28075,28175,28357,28506,28548,29026,29559,29638,30039,30447,30515,32070,32575,33497,33601,35118,35409,36133,37709,38740,39446,39758,41707,42031,42142,42466,43710,44061,44692,44964,2202,4522,18395,22497,45316],axis = 0, inplace = True)

In [ ]:
test_t.drop(labels = [1727,5031,853,1107,1432,2863,3214,4197,4837,4868],axis = 0, inplace = True)

In [ ]:
from datasets import Dataset

data_train = Dataset.from_pandas(train_t)
data_test = Dataset.from_pandas(test_t)

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(data_train)

,context,context_roles,question,question_roles,context_label,question_label,answers,__index_level_0__
0,"По классификации, опубликованной в издании Болгарская диалектология (под редакцией С. Стойкова), восточноболгарские говоры подразделяются на мизийские, балканские и рупские, а западноболгарские говоры — на северо-западные, юго-западные и крайнезападные. Согласно диалектологической карте болгарского языка, опубликованной отделением диалектологии и лингвистической географии Института болгарского языка, восточноболгарские говоры подразделяют на северо-восточные и юго-восточные, а западноболгарские говоры — на северо-западные и юго-западные.","<unk>,<unk>,<unk>,<pred>,<unk>,способ,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,пациенс,<unk>,<unk>,<unk>,тема,<pred>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,результат,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,агенс,<unk>,<unk>,<unk>,<pred>,способ,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<pred>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>","По какой классификации восточноболгарские говоры подразделяются на мизийские, балканские и рупские?","<unk>,<unk>,тема,<unk>,пациенс,<pred>,<unk>,результат,<unk>,<unk>,<unk>,<unk>,<unk>","[1, 1, 1, 0, 1, 29, 1, 1, 1, 1, 1, 1, 1, 14, 1, 1, 1, 41, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 24, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 29, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 41, 1, 14, 0, 1, 24, 1, 1, 1, 1, 1]","{'answer_start': [0], 'text': ['По классификации, опубликованной в издании Болгарская диалектология ']}",9150
1,"Позвонки, из которых построен позвоночный столб, у разных групп животных имеют различное строение. У высших позвоночных между телами позвонков располагаются хрящевые прослойки — межпозвоночные диски. Они выполняют функцию амортизаторов и обеспечивают подвижность позвоночника. Каждый позвонок состоит из тела и дуги, которые ограничивают позвоночное отверстие, находящееся между ними, а также отростков — остистого, поперечных и суставных. Верхние дуги позвонков окружают спинной мозг, отростки сочленяют позвонки между собой, к ним прикрепляются мышцы. Вместе отверстия позвонков формируют позвоночный канал, являющийся вместилищем спинного мозга.","тема,<unk>,<unk>,начальная точка,<pred>,<unk>,результат,<unk>,<unk>,<unk>,место,<unk>,<pred>,<unk>,результат,<unk>,<unk>,<unk>,<unk>,<unk>,начальная точка,место,<pred>,<unk>,<unk>,<unk>,<unk>,пациенс,<unk>,агенс,<pred>,пациенс,<unk>,<unk>,<pred>,пациенс,<unk>,<unk>,<unk>,тема,<pred>,<unk>,<unk>,<unk>,тема,<unk>,эффектор,<pred>,<unk>,пациенс,<unk>,<pred>,<unk>,агенс,<unk>,<unk>,<unk>,тема,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,пациенс,<pred>,<unk>,пациенс,<unk>,<unk>,<pred>,<unk>,<unk>,<unk>,<unk>,<unk>,место,<pred>,тема,<unk>,<unk>,агенс,<unk>,<pred>,<unk>,пациенс,<unk>,<pred>,статус,<unk>,<unk>,<unk>",Что окружают верхние дуги позвонков?,"тема,<pred>,<unk>,пациенс,<unk>,<unk>","[41, 1, 1, 13, 0, 1, 24, 1, 1, 1, 12, 1, 0, 1, 24, 1, 1, 1, 1, 1, 13, 12, 0, 1, 1, 1, 1, 14, 1, 2, 0, 14, 1, 1, 0, 14, 1, 1, 1, 41, 0, 1, 1, 1, 41, 1, 45, 0, 1, 14, 1, 0, 1, 2, 1, 1, 1, 41, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 0, 1, 14, 1, 1, 0, 1, 1, 1, 1, 1, 12, 0, 41, 1, 1, 2, 1, 0, 1, 14, 1, 0, 31, 1, 1, 1]","[41, 0, 1, 14, 1, 1]","{'answer_start': [472], 'text': ['спинной мозг']}",19715
2,"Однако в то время были и натуралисты, которые размышляли об эволюционном изменении организмов, происходящем в течение длительного времени. Мопертюи писал в 1751 году о естественных модификациях, происходящих во время воспроизводства, накапливающихся в течение многих поколений и приводящих к формированию новых видов. Бюффон предположил, что виды могут дегенерировать и превращаться в другие организмы. Эразм Дарвин считал, что все теплокровные организмы возможно происходят от одного микроорганизм

## Предобработка тренировочных данных

Для токеницизации данных мы будем использовать модели sbert_large_nlu_ru и deep_pavlov

In [ ]:
from transformers import AutoTokenizer

sber = "sberbank-ai/sbert_large_nlu_ru"
deep_pavlov="DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(deep_pavlov)
# tokenizer = AutoTokenizer.from_pretrained(sber)

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Првоерим токеницизацию

In [ ]:
test = tokenizer("Как меня зовут?", "Меня зовут - Вашингтон.")

In [ ]:
test_tokens = tokenizer.convert_ids_to_tokens(test["input_ids"])
print(test_tokens)

['[CLS]', 'Как', 'меня', 'зовут', '?', '[SEP]', 'Меня', 'зовут', '-', 'Вашингтон', '.', '[SEP]']


In [ ]:
test_1 = tokenizer("Чем заболел Байрон в Миссолонге?")
test_1['input_ids']

[101, 10944, 40861, 86762, 845, 18469, 59720, 6625, 166, 102]

In [ ]:
[0, 1, 2, 13, 1, 25, 0]

[0, 1, 2, 13, 1, 25, 0]

In [ ]:
test_tokens = tokenizer.convert_ids_to_tokens(test_1["input_ids"])
print(test_tokens)

['[CLS]', 'Чем', 'заболел', 'Байрон', 'в', 'Мисс', '##олон', '##ге', '?', '[SEP]']


In [ ]:
test_1.word_ids()

[None, 0, 1, 2, 3, 4, 4, 4, 5, None]

In [ ]:
test_1

{'input_ids': [101, 10944, 40861, 86762, 845, 18469, 59720, 6625, 166, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
max_length = 512
doc_stride = 128

In [ ]:
example = data_train[16]

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
)

In [ ]:
tokenized_example

{'input_ids': [[101, 5405, 34106, 86762, 70401, 13141, 166, 102, 31392, 2882, 128, 845, 5785, 9551, 86762, 851, 1641, 5785, 1699, 23480, 1506, 2752, 11384, 39081, 128, 25605, 20562, 2752, 29605, 18948, 132, 10795, 2886, 9551, 120, 1469, 44123, 130, 65683, 845, 15262, 128, 6350, 5746, 22565, 1768, 122, 128, 2752, 12802, 4745, 20335, 89075, 16144, 128, 625, 14673, 106735, 1703, 9439, 869, 53445, 19200, 132, 43085, 86762, 58461, 845, 99042, 842, 128, 851, 4546, 22742, 1519, 15511, 71289, 1436, 128, 2739, 4912, 70401, 13141, 128, 2067, 32198, 1469, 1574, 845, 44359, 12245, 128, 6575, 22059, 845, 54532, 30383, 132, 805, 24986, 7460, 5117, 94713, 49607, 6379, 45946, 132, 102007, 24564, 128, 118314, 858, 9063, 7526, 45761, 94713, 128, 54441, 128, 1997, 67877, 1519, 7021, 18716, 869, 9680, 128, 1997, 45487, 128, 3435, 2752, 14673, 12929, 36926, 1703, 6678, 1641, 2752, 60221, 21500, 2900, 851, 15537, 2599, 845, 26763, 1997, 43950, 132, 2988, 50760, 7473, 2886, 21310, 35920, 52915, 6887, 128, 34

In [ ]:
[len(x) for x in tokenized_example["input_ids"]]

[338]

In [ ]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] Как леди Байрон называла сына? [SEP] Бедность, в которой родился Байрон и от которой не избавил его титул лорда, дала направление его будущей карьере. Когда он родился ( на Холл - Стрит в Лондоне, 22 января 1788 года ), его отец уже потерял семейное состояние, а мать возвратилась из Европы с остатками состояния. Леди Байрон поселилась в Абердине, и её хромой мальчуган, как она называла сына, был отдан на год в частную школу, затем переведён в классическую гимназию. О детских выходках Байрона рассказывают много историй. Сестры Грей, нянчившие маленького Байрона, находили, что лаской можно делать с ним, что угодно, но его мать всегда выходила из себя от его непослушания и бросала в мальчика что попало. На вспышки матери он нередко отвечал насмешками, но однажды, как он сам рассказывает, у него отняли нож, которым он хотел заколоть себя. В гимназии он учился плохо, и Мэри Грей, читавшая ему псалмы и Библию, принесла ему больше пользы, чем гимназические учителя. Когда Джорджу исполни

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride,
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 8), (9, 15), (16, 24), (25, 29), (29, 30), (0, 0), (0, 3), (3, 8), (8, 9), (10, 11), (12, 19), (20, 27), (28, 34), (35, 36), (37, 39), (40, 47), (48, 50), (51, 56), (56, 58), (59, 62), (63, 68), (69, 74), (74, 75), (76, 80), (81, 92), (93, 96), (97, 104), (105, 112), (112, 113), (114, 119), (120, 122), (123, 130), (131, 132), (132, 134), (135, 139), (139, 140), (140, 145), (146, 147), (148, 155), (155, 156), (157, 159), (160, 166), (167, 171), (172, 176), (176, 177), (177, 178), (179, 182), (183, 187), (188, 191), (192, 199), (200, 208), (209, 218), (218, 219), (220, 221), (222, 226), (227, 239), (240, 242), (243, 249), (250, 251), (252, 261), (262, 271), (271, 272), (273, 277), (278, 284), (285, 295), (296, 297), (298, 305), (305, 306), (306, 307), (308, 309), (310, 312), (313, 317), (317, 319), (320, 324), (324, 327), (327, 329), (330, 331), (332, 335), (336, 339), (340, 348), (349, 353), (353, 354), (355, 358), (359, 364), (365, 367), (368, 371), (372, 373), (37

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0] : offsets[1]])

Как Как


In [ ]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["context"], truncation=True)

    labels = []
    for i, label in enumerate(examples["context_label"]):

        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx == len(label):
              label_ids.append(1)
              label_ids.append(1)
              break
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                # print(word_idx)
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
label_all_tokens = True

def tokenize_and_align_labels_c(examples):
    tokenized_inputs = tokenizer(examples["context"], truncation=True)

    labels = []
    for i, label in enumerate(examples["context_label"]):

        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx == len(label):
              label_ids.append(1)
              label_ids.append(-100)
              break
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                # print(word_idx)
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    return labels

In [ ]:
def tokenize_and_align_labels_q(examples):
    tokenized_inputs = tokenizer(examples["question"], truncation=True)

    labels = []
    for i, label in enumerate(examples["question_label"]):

        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx == len(label):
              label_ids.append(1)
              label_ids.append(-100)
              break
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                # print(word_idx)
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    return labels

In [ ]:
tests = tokenizer(data_train[2]['context'])
tests_l = data_train[2]['context_label']

In [ ]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] Как леди Байрон называла сына? [SEP] Бедность, в которой родился Байрон и от которой не избавил его титул лорда, дала направление его будущей карьере. Когда он родился ( на Холл - Стрит в Лондоне, 22 января 1788 года ), его отец уже потерял семейное состояние, а мать возвратилась из Европы с остатками состояния. Леди Байрон поселилась в Абердине, и её хромой мальчуган, как она называла сына, был отдан на год в частную школу, затем переведён в классическую гимназию. О детских выходках Байрона рассказывают много историй. Сестры Грей, нянчившие маленького Байрона, находили, что лаской можно делать с ним, что угодно, но его мать всегда выходила из себя от его непослушания и бросала в мальчика что попало. На вспышки матери он нередко отвечал насмешками, но однажды, как он сам рассказывает, у него отняли нож, которым он хотел заколоть себя. В гимназии он учился плохо, и Мэри Грей, читавшая ему псалмы и Библию, принесла ему больше пользы, чем гимназические учителя. Когда Джорджу исполни

None - специальные символы, 0 - вопрос, 1 - контекст

In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Находим начальный и конечный индекс ответа

In [ ]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

73 77


Проверяем что индексы найдены верно

In [ ]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position : end_position + 1]))
print(answers["text"][0])

хромой мальчуган
хромой мальчуган


## Токенизация

Метод для растягивания Labels

In [ ]:
label_all_tokens = True
max_length = 512
doc_stride = 128

# [None, 0, 1, 2, 2, 2, 3, 4, 5, None, 0, 1, 2, 3,
 

def tokenize_and_align_labels_c(tokenized_inputs, examples):

    # labels_q = []
    labels = []
    for i, label in enumerate(examples["question_label"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        q_len = 0

        for j, wr in enumerate(word_ids):
          if wr is None and j != 0:
            q_len = j
            break

        word_ids_q = word_ids[:q_len+1]
        for word_idx in word_ids_q:
            if word_idx == len(label):
              label_ids.append(1)
              break
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(46)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                # print(word_idx)
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else 46)
            previous_word_idx = word_idx
        
        # print(label_ids)
        
        context_label = examples["context_label"][i]

        # word_ids = tokenized_inputs.word_ids(batch_index=i)

        # # for j, wr in enumerate(word_ids):
        # #   if wr is None and j != 0:
        # #     word_ids = word_ids[j+1:]
        # #     break

        word_ids_c = word_ids[q_len+1:]

        for word_idx in word_ids_c:
            
            if word_idx == len(context_label):
              label_ids.append(1)
              label_ids.append(46)
              break
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(46)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                try:
                  label_ids.append(context_label[word_idx])
                except IndexError as e:
                  print(example['context'])
                  # print(i)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
              try:
                label_ids.append(context_label[word_idx] if label_all_tokens else 46)
              except IndexError as e:
                print(example['context'])
            previous_word_idx = word_idx

        lenn = len(label_ids)
        while lenn != 512:
          label_ids.append(46)
          lenn += 1
        # print(len(label_ids))
        labels.append(label_ids) 


    return labels

Метод для токенизация всех данных

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

def prepare_train_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # print((tokenized_examples.sequence_ids()))
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # print(len(sample_mapping))
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    tokenized_examples["labels"] = tokenize_and_align_labels_c(tokenized_examples, examples)
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # print(tokenized_examples.word_ids())
        # print(i)
        sequence_ids = tokenized_examples.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # if i + 1 < len(offset_mapping):
        #   if sample_mapping[i + 1] == sample_index:

        #     print(sample_index)

        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)
                
    # tokenized_examples['context_label'] = tokenize_and_align_labels_c(examples)
    # tokenized_examples['question_label'] = tokenize_and_align_labels_q(examples)
    return tokenized_examples

И применяем нашу функцию ко всем данным

In [ ]:
tokenized_test = data_test.map(prepare_train_features, batched=True, remove_columns=data_test.column_names)

tokenized_train = data_train.map(prepare_train_features, batched=True, remove_columns=data_train.column_names)



Parameter 'function'=<function prepare_train_features at 0x7fde835be3b0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/46 [00:00<?, ?ba/s]

In [ ]:
tokenized_train

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'labels'],
    num_rows: 45226
})

## Обучение

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(sber)

Downloading:   0%|          | 0.00/1.59G [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias', 'emb.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(deep_pavlov)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from 

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer


# model = AutoModelForQuestionAnswering.from_pretrained('ruselkomp/sber-full-test')
# model = AutoModelForQuestionAnswering.from_pretrained("ruselkomp/sber-full-framebank")
# model = AutoModelForQuestionAnswering.from_pretrained("ruselkomp/deepavlov-test-bert-2")
model = AutoModelForQuestionAnswering.from_pretrained("ruselkomp/deep-pavlov-full-2")


Downloading:   0%|          | 0.00/869 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/676M [00:00<?, ?B/s]

In [ ]:
batch_size = 16

In [ ]:
model_name = sber.split("/")[-1]
args = TrainingArguments(
    f"ruselkomp/sber-framebank-50size",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    save_steps = 4000,
)

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

Cloning https://huggingface.co/ruselkomp/sber-framebank-50size-tetsss into local empty directory.


In [ ]:
trainer.train('checkpoint-20000')

Loading model from checkpoint-20000).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 45226
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 33921
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 1
  Continuing training from global step 20000
  Will skip the first 1 epochs then the first 8693 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/8693 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss
2,0.814500,1.177848
3,0.616800,1.373564


***** Running Evaluation *****
  Num examples = 5022
  Batch size = 4
Saving model checkpoint to ruselkomp/sber-framebank-50size-2/checkpoint-24000
Configuration saved in ruselkomp/sber-framebank-50size-2/checkpoint-24000/config.json
Model weights saved in ruselkomp/sber-framebank-50size-2/checkpoint-24000/pytorch_model.bin
tokenizer config file saved in ruselkomp/sber-framebank-50size-2/checkpoint-24000/tokenizer_config.json
Special tokens file saved in ruselkomp/sber-framebank-50size-2/checkpoint-24000/special_tokens_map.json
tokenizer config file saved in ruselkomp/sber-framebank-50size-2/tokenizer_config.json
Special tokens file saved in ruselkomp/sber-framebank-50size-2/special_tokens_map.json
Saving model checkpoint to ruselkomp/sber-framebank-50size-2/checkpoint-28000
Configuration saved in ruselkomp/sber-framebank-50size-2/checkpoint-28000/config.json
Model weights saved in ruselkomp/sber-framebank-50size-2/checkpoint-28000/pytorch_model.bin
tokenizer config file saved in rusel

TrainOutput(global_step=33921, training_loss=0.2661936727251486, metrics={'train_runtime': 21044.2641, 'train_samples_per_second': 6.447, 'train_steps_per_second': 1.612, 'total_flos': 1.2600514895822438e+17, 'train_loss': 0.2661936727251486, 'epoch': 3.0})

In [ ]:
trainer.save_model("sber-framebank-50size")

Saving model checkpoint to sber-framebank-50size
Configuration saved in sber-framebank-50size/config.json
Model weights saved in sber-framebank-50size/pytorch_model.bin
tokenizer config file saved in sber-framebank-50size/tokenizer_config.json
Special tokens file saved in sber-framebank-50size/special_tokens_map.json
Saving model checkpoint to ruselkomp/sber-framebank-50size-2
Configuration saved in ruselkomp/sber-framebank-50size-2/config.json
Model weights saved in ruselkomp/sber-framebank-50size-2/pytorch_model.bin
tokenizer config file saved in ruselkomp/sber-framebank-50size-2/tokenizer_config.json
Special tokens file saved in ruselkomp/sber-framebank-50size-2/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/1.59G [00:00<?, ?B/s]

Upload file runs/May31_10-02-56_2a406c9c041f/events.out.tfevents.1653991430.2a406c9c041f.2401.0: 100%|########…

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/ruselkomp/sber-framebank-50size-2
   f89c9d2..1814591  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/ruselkomp/sber-framebank-50size-2
   1814591..6dfa29f  main -> main



In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 45226
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 33921


Epoch,Training Loss,Validation Loss
1,1.060100,1.084853
2,0.691800,1.158811
3,0.407100,1.567183


Saving model checkpoint to tests-finetuned-squad-full/checkpoint-2000
Configuration saved in tests-finetuned-squad-full/checkpoint-2000/config.json
Model weights saved in tests-finetuned-squad-full/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in tests-finetuned-squad-full/checkpoint-2000/tokenizer_config.json
Special tokens file saved in tests-finetuned-squad-full/checkpoint-2000/special_tokens_map.json
tokenizer config file saved in tests-finetuned-squad-full/tokenizer_config.json
Special tokens file saved in tests-finetuned-squad-full/special_tokens_map.json
Saving model checkpoint to tests-finetuned-squad-full/checkpoint-4000
Configuration saved in tests-finetuned-squad-full/checkpoint-4000/config.json
Model weights saved in tests-finetuned-squad-full/checkpoint-4000/pytorch_model.bin
tokenizer config file saved in tests-finetuned-squad-full/checkpoint-4000/tokenizer_config.json
Special tokens file saved in tests-finetuned-squad-full/checkpoint-4000/special_tokens_m

TrainOutput(global_step=33921, training_loss=0.8008415258379534, metrics={'train_runtime': 32441.5314, 'train_samples_per_second': 4.182, 'train_steps_per_second': 1.046, 'total_flos': 1.260051156139991e+17, 'train_loss': 0.8008415258379534, 'epoch': 3.0})

## Проверка модели

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [ ]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 512]), torch.Size([16, 512]))

In [ ]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 16,  39,  27,  34, 106,  81,  57,  16, 113,  61, 135,  92, 132,  45,
          41, 102], device='cuda:0'),
 tensor([ 16,  41,  28,  35, 107,  82,  63,  17, 116,  64, 143, 100, 134,  47,
          47, 107], device='cuda:0'))

Выбираем 20 лучших вариантов ответа

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()

start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index:
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": ""
                }
            )

Сортируем ответы по их оценке (SCORE) и проверяем что наш ответ точно находится в контексте, а не в вопросе

In [ ]:
def prepare_validation_features(examples):

    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    tokenized_examples["example_id"] = []
    tokenized_examples["labels"] = tokenize_and_align_labels_c(tokenized_examples, examples)
    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["__index_level_0__"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
validation_features = data_test.map(
    prepare_validation_features,
    batched=True,
    remove_columns=data_test.column_names
)

  0%|          | 0/6 [00:00<?, ?ba/s]

In [ ]:
validation_features

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id', 'labels'],
    num_rows: 5022
})

In [ ]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5022
  Batch size = 16


In [ ]:
print((raw_predictions[0][0][0]))

[-5.728 -6.781 -7.398 -7.856 -7.653 -7.874 -8.237 -8.358 -7.949 -7.015
 -2.764 -3.575 -3.656 -4.018  3.413  4.031  7.781 -4.145 -3.464 -5.82
 -6.166 -6.141 -6.025 -6.34  -5.329 -6.029 -5.915 -5.485 -3.96  -4.062
 -6.993 -4.876 -7.457 -6.534 -7.026 -7.198 -7.951 -6.858 -6.129 -7.055
 -7.766 -7.528 -7.072 -8.18  -7.175 -5.904 -7.701 -7.891 -6.857 -7.234
 -7.561 -7.885 -8.007 -6.84  -7.261 -7.556 -7.44  -7.431 -7.567 -7.84
 -8.227 -8.254 -7.398 -7.559 -7.462 -7.667 -7.966 -8.208 -7.337 -7.106
 -7.802 -7.658 -6.978 -7.498 -8.067 -8.234 -8.262 -7.459 -7.225 -7.457
 -7.693 -7.879 -8.628 -7.766 -7.752 -7.223 -7.639 -8.021 -7.654 -7.713
 -8.085 -7.332 -6.529 -7.556 -7.493 -6.692 -7.191 -7.452 -6.949 -8.172
 -7.78  -6.808 -7.201 -7.804 -7.728 -6.344 -7.2   -7.347 -7.969 -7.195
 -7.426 -7.478 -7.872 -8.182 -7.546 -7.964 -7.999 -7.999 -8.189 -8.331
 -8.047 -8.038 -7.829 -8.589 -8.37  -7.009 -7.436 -7.788 -7.865 -8.227
 -8.035 -8.02  -8.1   -8.619 -8.285 -7.022 -7.787 -7.764 -7.457 -7.546
 -7.938 

In [ ]:
max_answer_length = 50

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]

context = data_test[0]["context"]

start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index:
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 15.076738, 'text': 'лихорадкой'},
 {'score': 13.927921, 'text': 'лихорадкой,'},
 {'score': 11.326551, 'text': 'заболел лихорадкой'},
 {'score': 10.708677, 'text': 'Байрон заболел лихорадкой'},
 {'score': 10.177734, 'text': 'заболел лихорадкой,'},
 {'score': 9.55986, 'text': 'Байрон заболел лихорадкой,'},
 {'score': 9.095023,
  'text': 'лихорадкой, продолжая отдавать все свои силы на борьбу за свободу страны.'},
 {'score': 8.297618,
  'text': 'лихорадкой, продолжая отдавать все свои силы на борьбу за свободу страны'},
 {'score': 5.3448353,
  'text': 'заболел лихорадкой, продолжая отдавать все свои силы на борьбу за свободу страны.'},
 {'score': 4.726962,
  'text': 'Байрон заболел лихорадкой, продолжая отдавать все свои силы на борьбу за свободу страны.'},
 {'score': 4.5474305,
  'text': 'заболел лихорадкой, продолжая отдавать все свои силы на борьбу за свободу страны'},
 {'score': 4.531169, 'text': 'В Миссолонги Байрон заболел лихорадкой'},
 {'score': 3.9295568,
  'text': 'Ба

In [ ]:
data_test[0]["answers"]

{'answer_start': [28], 'text': ['лихорадкой']}

In [ ]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["__index_level_0__"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]

            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["__index_level_0__"]] = best_answer["text"]

    return predictions


In [ ]:
import collections
from tqdm.auto import tqdm

examples = data_test
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["__index_level_0__"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    
final_predictions = postprocess_qa_predictions(data_test, validation_features, raw_predictions.predictions)
references = [{"qas_id": ex["__index_level_0__"], "answers": ex["answers"]['text'][0]} for ex in data_test]

Post-processing 5022 example predictions split into 5022 features.


  0%|          | 0/5022 [00:00<?, ?it/s]

In [ ]:
res = pd.DataFrame(references)
res['prediction'] = res['qas_id'].apply(lambda r: final_predictions[r])

In [ ]:
list_true = res['answers'].tolist()
list_pred = res['prediction'].tolist()

In [ ]:
import re
import string
from collections import Counter
from typing import List

from deeppavlov.core.common.metrics_registry import register_metric


def squad_v1_f1(y_true: List[List[str]], y_predicted: List[str]) -> float:
    """ Calculates F-1 score between y_true and y_predicted
        F-1 score uses the best matching y_true answer
        Skips examples without an answer.
    Args:
        y_true: list of correct answers (correct answers are represented by list of strings)
        y_predicted: list of predicted answers
    Returns:
        F-1 score : float
    """
    f1_total = 0.0
    count = 0
    for ground_truth, prediction in zip(y_true, y_predicted):
        if len(ground_truth[0]) == 0:
            # skip empty answers
            continue
        count += 1
        prediction_tokens = normalize_answer(prediction).split()
        f1s = []
        ground_truth_tokens = normalize_answer(ground_truth).split()
        common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
        num_same = sum(common.values())
        if num_same == 0:
            f1s.append(0.0)
            continue
        precision = 1.0 * num_same / len(prediction_tokens)
        recall = 1.0 * num_same / len(ground_truth_tokens)
        f1 = (2 * precision * recall) / (precision + recall)
        f1s.append(f1)
        f1_total += max(f1s)
    return 100 * f1_total / count if count > 0 else 0

def squad_v1_exact_match(y_true: List[List[str]], y_predicted: List[str]) -> float:
    """ Calculates Exact Match score between y_true and y_predicted
        EM score uses the best matching y_true answer:
            if y_pred equal at least to one answer in y_true then EM = 1, else EM = 0
        Skips examples without an answer.
    Args:
        y_true: list of correct answers (correct answers are represented by list of strings)
        y_predicted: list of predicted answers
    Returns:
        exact match score : float
    """
    EM_total = 0
    count = 0
    for ground_truth, prediction in zip(y_true, y_predicted):
        if len(ground_truth[0]) == 0:
            # skip empty answers
            continue
        # print(ground_truth)
        # print(prediction)
        # print('\n')
        count += 1
        EMs = [int(normalize_answer(ground_truth) == normalize_answer(prediction))]
        EM_total += max(EMs)
    return 100 * EM_total / count if count > 0 else 0


def normalize_answer(s: str) -> str:
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
def get_f1_em(list_true, list_pred, model_name):
  f1 = squad_v1_f1(list_true, list_pred)
  em = squad_v1_exact_match(list_true, list_pred)
  print(model_name)
  print("F1: %f\nEM: %f"%(f1,em))

## Результаты

Sber Large

In [ ]:
get_f1_em(list_true, list_pred, 'Sber')

Sber
F1: 83.469967
EM: 63.580247


Sber Large + Label Size 10

In [ ]:
get_f1_em(list_true, list_pred, 'Sber + Framebank')

Sber + Framebank
F1: 83.384670
EM: 63.480685


DeepPavlov

In [ ]:
get_f1_em(list_true, list_pred, 'DeepPavlov')

DeepPavlov
F1: 83.991671
EM: 63.998407


DeepPavlov + Label Size 10

In [ ]:
get_f1_em(list_true, list_pred, 'DeepPavlov + Framebank')

DeepPavlov + Framebank
F1: 83.628515
EM: 63.381123


## Тесты с различными длинами Labels

In [ ]:
get_f1_em(list_true, list_pred, 'DeepPavlov')

DeepPavlov
F1: 83.991671
EM: 63.998407


LabelSize = 10, epoch = 5

In [ ]:
get_f1_em(list_true, list_pred, 'DeepPavlov + Framebank + 5epoch')

DeepPavlov + Framebank + 5epoch
F1: 83.322322
EM: 62.086818


LabelSize = TokenSize / 2

In [ ]:
#ruselkomp/deep-pavlov-framebank-hidesize
get_f1_em(list_true, list_pred, 'DeepPavlov + Framebank + HideSize / 2')

DeepPavlov + Framebank + HideSize
F1: 83.657054
EM: 63.859020


LabelSize = TokenSize

In [ ]:
#ruselkomp/deep-pavlov-framebank-hidesize-1
get_f1_em(list_true, list_pred, 'DeepPavlov + Framebank + HideSize (full)')

DeepPavlov + Framebank + HideSize (full)
F1: 84.095205
EM: 64.317005


In [ ]:
#ruselkomp/sber-framebank-hidesize-1
get_f1_em(list_true, list_pred, 'DeepPavlov + Framebank + Sber (full)')

DeepPavlov + Framebank + Sber (full)
F1: 82.589119
EM: 62.504978


LabelSize = TokenSize * 0.75

In [ ]:
#ruselkomp/sber-framebank-hidesize-2
get_f1_em(list_true, list_pred, 'DeepPavlov + Framebank + Sber (0.75 * full)')

DeepPavlov + Framebank + Sber (0.75 * full)
F1: 83.574943
EM: 63.142174


LabelSize = TokenSize / 6

In [ ]:
get_f1_em(list_true, list_pred, 'DeepPavlov + Framebank + HideSize (full / 6)')

DeepPavlov + Framebank + HideSize (full / 6)
F1: 83.747137
EM: 64.137794


LabelSize = TokenSize , epoch = 5

In [ ]:
#ruselkomp/deeppavlov-framebank-full-5epochs
get_f1_em(list_true, list_pred, 'DeepPavlov + Framebank + full + 5 epochs')

DeepPavlov + Framebank + full + 5 epochs
F1: 83.121687
EM: 61.907607


LabelSize = 50

In [ ]:
#ruselkomp/deeppavlov-framebank-full-5epochs
get_f1_em(list_true, list_pred, 'DeepPavlov + Framebank + HideSize (50)')

DeepPavlov + Framebank + HideSize (50)
F1: 84.070120
EM: 64.376742


In [ ]:
#ruselkomp/deeppavlov-framebank-full-5epochs
get_f1_em(list_true, list_pred, 'Sber + Framebank + HideSize (50)')

Sber + Framebank + HideSize (50)
F1: 82.855885
EM: 62.425329
